In [1]:
# Import required packages
import os
import sys
import getpass
#import asf_search as asf
import logging
import isce
root_logger = logging.getLogger()
root_logger.setLevel('WARNING')
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from datetime import datetime, timedelta
import time
from glob import glob

In [2]:
# Set environment variables to call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['ISCE_STACK'] = '/home/jovyan/tools/isce2/src/isce2/contrib/stack'
os.environ['PYTHONPATH'] = os.environ['ISCE_STACK']
os.environ['PATH'] += f":{os.environ['ISCE_STACK']}/topsStack"

## define utilities

## download data

In [3]:
scene_list=['S1A_IW_SLC__1SDV_20201006T001135_20201006T001202_034668_0409B2_0D91',
            'S1A_IW_SLC__1SDV_20201018T001133_20201018T001159_034843_040FD6_7A19',
            'S1A_IW_SLC__1SDV_20201030T001133_20201030T001159_035018_0415C8_9680',
            'S1A_IW_SLC__1SDV_20201111T001132_20201111T001159_035193_041BE5_06B2',
            'S1A_IW_SLC__1SDV_20201123T001132_20201123T001159_035368_0421EE_C267',
            'S1A_IW_SLC__1SDV_20201205T001132_20201205T001158_035543_0427F0_9243',
            'S1A_IW_SLC__1SDV_20201217T001131_20201217T001158_035718_042E06_027E',
            'S1A_IW_SLC__1SDV_20201229T001131_20201229T001157_035893_04341E_E3B4'
           ]

In [ ]:
EARTHDATA_LOGIN = "qbrencherUW"
EARTHDATA_PASSWORD = getpass.getpass()

In [4]:
proc_path = '/home/jovyan/ffits/nbs/imja/proc_DT121'

In [ ]:
# download slcs
os.chdir(f'{proc_path}/slc')

results = asf.granule_search(scene_list)
session = asf.ASFSession().auth_with_creds(EARTHDATA_LOGIN, EARTHDATA_PASSWORD)
results.download(path=f'{proc_path}/slc', processes=2, session=session)

In [ ]:
# download aux
os.chdir(f'{proc_path}/aux')
!wget https://sar-mpc.eu/download/ca97845e-1314-4817-91d8-f39afbeff74d/ -O S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.zip

In [ ]:
os.chdir(f'{proc_path}/dem')

In [ ]:
%%bash
dem.py -a stitch -b 27 29 86 88 -r -s 1 -c
rm demLat*.dem demLat*.dem.xml demLat*.dem.vrt

## run topsStack

In [5]:
os.chdir(proc_path)

In [6]:
!stackSentinel.py -s ./slc -o ./orbits -a ./aux -d ./dem/demLat_N27_N29_Lon_E086_E088.dem.wgs84 -w ./work -C geometry --bbox '27.80 28.13 86.28 87.32' -W interferogram -c 3 --azimuth_looks 3 --range_looks 7

Number of SAFE files found: 8
*****************************************
20201006
orbit was not found in the /home/jovyan/ffits/nbs/imja/proc_DT121/orbits
downloading precise or restituted orbits ...
fetchOrbit.py -i /home/jovyan/ffits/nbs/imja/proc_DT121/slc/S1A_IW_SLC__1SDV_20201006T001135_20201006T001202_034668_0409B2_0D91.zip -o /home/jovyan/ffits/nbs/imja/proc_DT121/work/orbits/20201006
Reference time:  2020-10-06 00:12:02
Satellite name:  S1A
*****************************************
20201018
orbit was not found in the /home/jovyan/ffits/nbs/imja/proc_DT121/orbits
downloading precise or restituted orbits ...
fetchOrbit.py -i /home/jovyan/ffits/nbs/imja/proc_DT121/slc/S1A_IW_SLC__1SDV_20201018T001133_20201018T001159_034843_040FD6_7A19.zip -o /home/jovyan/ffits/nbs/imja/proc_DT121/work/orbits/20201018
Reference time:  2020-10-18 00:11:59
Satellite name:  S1A
*****************************************
20201030
orbit was not found in the /home/jovyan/ffits/nbs/imja/proc_DT121/orbits
do

In [7]:
os.chdir(f'{proc_path}/work/run_files')

In [8]:
%%time
!sh run_01_unpack_topo_reference

Completed Parsing the Configuration file
Functions to be executed:
['Function-1', 'Function-2']
Running: Sentinel1_TOPS
['--dirname', '/home/jovyan/ffits/nbs/imja/proc_DT121/slc/S1A_IW_SLC__1SDV_20201006T001135_20201006T001202_034668_0409B2_0D91.zip', '--swaths', '1 2 3', '--orbit', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/orbits/20201006/S1A_OPER_AUX_POEORB_OPOD_20210317T225626_V20201005T225942_20201007T005942.EOF', '--outdir', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/reference', '--auxdir', '/home/jovyan/ffits/nbs/imja/proc_DT121/aux', '--bbox', '27.80 28.13 86.28 87.32', '--pol', 'vv']
Input XML files:  ['/vsizip//home/jovyan/ffits/nbs/imja/proc_DT121/slc/S1A_IW_SLC__1SDV_20201006T001135_20201006T001202_034668_0409B2_0D91.zip/S1A_IW_SLC__1SDV_20201006T001135_20201006T001202_034668_0409B2_0D91.SAFE/annotation/s1a-iw1-slc-vv-20201006t001136-20201006t001201-034668-0409b2-004.xml']
Input TIFF files:  ['/vsizip//home/jovyan/ffits/nbs/imja/proc_DT121/slc/S1A_IW_SLC__1SDV_20201006T

In [9]:
%%time
!sh run_02_unpack_secondary_slc

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: Sentinel1_TOPS
['--dirname', '/home/jovyan/ffits/nbs/imja/proc_DT121/slc/S1A_IW_SLC__1SDV_20201018T001133_20201018T001159_034843_040FD6_7A19.zip', '--swaths', '1 2 3', '--orbit', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/orbits/20201018/S1A_OPER_AUX_POEORB_OPOD_20210318T024542_V20201017T225942_20201019T005942.EOF', '--outdir', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/secondarys/20201018', '--auxdir', '/home/jovyan/ffits/nbs/imja/proc_DT121/aux', '--bbox', '27.80 28.13 86.28 87.32', '--pol', 'vv']
Input XML files:  ['/vsizip//home/jovyan/ffits/nbs/imja/proc_DT121/slc/S1A_IW_SLC__1SDV_20201018T001133_20201018T001159_034843_040FD6_7A19.zip/S1A_IW_SLC__1SDV_20201018T001133_20201018T001159_034843_040FD6_7A19.SAFE/annotation/s1a-iw1-slc-vv-20201018t001133-20201018t001159-034843-040fd6-004.xml']
Input TIFF files:  ['/vsizip//home/jovyan/ffits/nbs/imja/proc_DT121/slc/S1A_IW_SLC__1SDV_20201018T0011

In [10]:
%%time
!sh run_03_average_baseline

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: computeBaseline
['--reference', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/reference/', '--secondary', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/secondarys/20201018', '--baseline_file', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/baselines/20201006_20201018/20201006_20201018.txt']
minSecondary,maxSecondary 1 2
minReference,maxReference 1 1
minBurst, maxBurst:  1 1
Polynomial Order: 0 - by - 0 
0	
Bprep:  [25.47092242605818]
Bpar:  [-13.432458399239861]
minSecondary,maxSecondary 1 3
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Bprep:  [24.714609208888753, 24.99753151871164]
Bpar:  [-15.45942259889572, -15.585127964303457]
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: computeBaseline
['--reference', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/reference/', '--secondary', '/

In [11]:
%%time
!sh run_04_fullBurst_geo2rdr

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: geo2rdr
['--secondary', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/secondarys/20201018', '--reference', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/reference', '--geom_referenceDir', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/geom_reference', '--coregSLCdir', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/coreg_secondarys/20201018', '--azimuth_misreg', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/misreg/azimuth/dates/20201018.txt', '--range_misreg', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/misreg/range/dates/20201018.txt']
CPU mode
Estimated burst offset:  1
Overlap pair 0: Burst 0 of reference matched with Burst 1 of secondary
Burst 0 of reference matched with Burst 1 of secondary
Additional time offset applied in geo2rdr: 0.0 secs
Additional range offset applied in geo2rdr: 0.0 m
Turning off bistatic delay correction term by default.
API open (WR): /home/jovyan/ffits/nbs/imja/proc_DT121/

In [12]:
%%time
!sh run_05_fullBurst_resample

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: resamp_withCarrier
['--secondary', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/secondarys/20201018', '--reference', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/reference', '--coregdir', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/coreg_secondarys/20201018', '--azimuth_misreg', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/misreg/azimuth/dates/20201018.txt', '--range_misreg', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/misreg/range/dates/20201018.txt']
Estimated burst offset:  1
Shifts:  {1: 0}
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Chi squared: 0.000026
Polynomial Order: 5 - by - 3 
13235.2	-637.956	28.2352	0.932118	
-51381.7	2474.12	-106.968	-3.58082	
49868.4	-2398.77	101.249	3.43864	
-3.10865e-10	-7.67277e-11	-1.05227e-10	0	
5.22777e-10	5.0136e-11	0	0	
-2.93312e-10	0	0	0	
Misfit radians - Max: 8.158355012710672e-05 , Min : -0.0001510170623078011 
True
2023-09-01 17

In [13]:
%%time
!sh run_06_extract_stack_valid_region

creating  /home/jovyan/ffits/nbs/imja/proc_DT121/work/stack
checking the number of bursts in coreg_secondarys against the one in reference
all secondary images have the same number of bursts as the reference
******************
swath:  1
/home/jovyan/ffits/nbs/imja/proc_DT121/work/coreg_secondarys/20201018
minSecondary,maxSecondary 1 1
minReference,maxReference 1 1
minBurst, maxBurst:  1 1
/home/jovyan/ffits/nbs/imja/proc_DT121/work/coreg_secondarys/20201030
minSecondary,maxSecondary 1 1
minReference,maxReference 1 1
minBurst, maxBurst:  1 1
/home/jovyan/ffits/nbs/imja/proc_DT121/work/coreg_secondarys/20201111
minSecondary,maxSecondary 1 1
minReference,maxReference 1 1
minBurst, maxBurst:  1 1
/home/jovyan/ffits/nbs/imja/proc_DT121/work/coreg_secondarys/20201123
minSecondary,maxSecondary 1 1
minReference,maxReference 1 1
minBurst, maxBurst:  1 1
/home/jovyan/ffits/nbs/imja/proc_DT121/work/coreg_secondarys/20201205
minSecondary,maxSecondary 1 1
minReference,maxReference 1 1
minBurst, max

In [14]:
%%time
!sh run_07_merge_reference_secondary_slc

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--stack', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/stack', '--inp_reference', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/reference', '--dirname', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/reference', '--name_pattern', 'burst*slc', '--outfile', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/SLC/20201006/20201006.slc', '--method', 'top', '--valid_only', '--use_virtual_files', '--range_looks', '7', '--azimuth_looks', '3']
Skipping processing of swath 1
Updating the valid region of each burst to the common valid region of the stack
bursts:  1 2
Skipping multi-looking ....
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--stack', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/stack', '--inp_reference', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/reference', '--dirname', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/geom

In [15]:
%%time
!sh run_08_generate_burst_igram

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: generateIgram
['--reference', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/reference', '--secondary', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/coreg_secondarys/20201018', '--interferogram', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/interferograms/20201006_20201018', '--interferogram_prefix', 'fine']
minSecondary,maxSecondary 1 1
minReference,maxReference 1 1
minBurst, maxBurst:  1 1
matching burst numbers:  1 1
multiply
read
No range offsets provided
Type:  <class 'isceobj.Sensor.TOPS.TOPSSwathSLCProduct.TOPSSwathSLCProduct'>
minSecondary,maxSecondary 1 2
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
matching burst numbers:  1 1
multiply
read
No range offsets provided
matching burst numbers:  2 2
multiply
read
No range offsets provided
Type:  <class 'isceobj.Sensor.TOPS.TOPSSwathSLCProduct.TOPSSwathSLCProduct'>
Completed Parsing the Configuration file
Functions to be executed:
['F

In [16]:
%%time
!sh run_09_merge_burst_igram

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--stack', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/stack', '--inp_reference', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/interferograms/20201006_20201018', '--dirname', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/interferograms/20201006_20201018', '--name_pattern', 'fine*int', '--outfile', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/fine.int', '--method', 'top', '--aligned', '--valid_only', '--use_virtual_files', '--multilook', '--range_looks', '7', '--azimuth_looks', '3']
Skipping processing of swath 1
Updating the valid region of each burst to the common valid region of the stack
bursts:  1 2
multilooking 7 x 3 using isce2 for /home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/fine.int.full ...
GDAL open (R): /home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/fine.int.

In [17]:
%%time
!sh run_10_filter_coherence

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: FilterAndCoherence
['--input', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/fine.int', '--filt', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/filt_fine.int', '--coh', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/filt_fine.cor', '--strength', '0.5', '--slc1', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/SLC/20201006/20201006.slc.full', '--slc2', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/SLC/20201018/20201018.slc.full', '--complex_coh', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/fine.cor', '--range_looks', '7', '--azimuth_looks', '3']
2023-09-01 17:55:01,241 - isce.tops.runFilter - INFO - Applying power-spectral filter
GDAL open (R): /home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/fine.i

In [18]:
%%time
!sh run_11_unwrap

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: unwrap
['--ifg', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/filt_fine.int', '--unw', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/filt_fine.unw', '--coh', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/filt_fine.cor', '--reference', '/home/jovyan/ffits/nbs/imja/proc_DT121/work/reference', '--defomax', '2', '--rlks', '7', '--alks', '3', '--method', 'snaphu']
unwrapping method :  snaphu

snaphu v1.4.2
Reading wrapped phase from file /home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/filt_fine.int
No weight file specified.  Assuming uniform weights
Reading correlation data from file /home/jovyan/ffits/nbs/imja/proc_DT121/work/merged/interferograms/20201006_20201018/filt_fine.cor
Calculating smooth-solution cost parameters
Initializing flows with MCF